In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import pdb

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True) 
from torch.utils.tensorboard import SummaryWriter


In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
import tensorboard
print(tensorboard.__version__)


1.1.0
0.3.0
1.15.0a20190806


In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels =6,kernel_size =5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels =12,kernel_size =5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
    # (1) input layer
        t = t

    # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

    # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

    # (6) output layer
        t = self.out(t)
    #t = F.softmax(t, dim=1)

        return t

In [6]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [7]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:  # Get Batch
        images, labels = batch 
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)


    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram(
        'conv1.weight.grad'
        , network.conv1.weight.grad
        , epoch
    )

    print(
        "epoch", epoch,
        "total_correct:", total_correct,
        "loss:", total_loss
    )

tb.close()

epoch 0 total_correct: 47762 loss: 326.8235500007868
epoch 1 total_correct: 51638 loss: 226.772524908185
epoch 2 total_correct: 52357 loss: 208.18916660547256
epoch 3 total_correct: 52735 loss: 200.1591150611639
epoch 4 total_correct: 52739 loss: 195.9166245907545
epoch 5 total_correct: 53007 loss: 190.7596720904112
epoch 6 total_correct: 53016 loss: 187.53476712107658
epoch 7 total_correct: 53105 loss: 186.38123197853565
epoch 8 total_correct: 53178 loss: 181.0451419055462
epoch 9 total_correct: 53131 loss: 185.5452545285225


In [8]:
network.named_parameters

<bound method Module.named_parameters of Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)>